## Create Data ver 3.1

@Juran 2021.11.30

- 미디어파이프 랜드마크 8-5번, 8-0번, 7-0번 사이 거리 측정하여 데이터셋 생성
- 미디어파이프 설치상의 문제로 google colab 이용
- colab에서 웹캠 사용 불가 문제로 영상 촬영하여 업로드 후 이용
- ver3.0 z좌표 고려 안함과 코드 오류 해결 -> 두 벡터사이 유클리드 거리 계산

### Google Colab File Upload & Install Library

In [7]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving click.mov to click.mov
Saving none.mov to none.mov
User uploaded file "click.mov" with length 40811357 bytes
User uploaded file "none.mov" with length 40523541 bytes


In [2]:
!pip install mediapipe

     |████████████████████████████████| 32.8 MB 1.5 MB/s 


### 라이브러리 import 및 미디어파이프 설정

In [10]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
import time
import math

In [11]:
from scipy.spatial import distance

In [4]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

### openCV & mediapipe 이용하여 랜드마크사이 거리 정보 추출

#### 클릭 모션

In [13]:
cap = cv2.VideoCapture('click.mov') # video open

data_click_ver3_1 = pd.DataFrame()

while cap.isOpened():
  ret, img = cap.read()
  if img is None:
    break
  img = cv2.flip(img, 1)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  result = hands.process(img)
  img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

  index_point = [[8, 5], [8, 0], [7, 0]]
  point_info = np.zeros((21, 3))
  if result.multi_hand_landmarks is not None:
    for res in result.multi_hand_landmarks:
      
      for j, lm in enumerate(res.landmark):
        point_info[j] = [lm.x, lm.y, lm.z]
      
      lengths = []
      for point in index_point:
        # 두 점 사이 거리 계산
        a = (point_info[point[0]][0], point_info[point[0]][1], point_info[point[0]][2]) # point[0]
        b = (point_info[point[1]][0], point_info[point[1]][1], point_info[point[1]][2]) # point[1]
        length = distance.euclidean(a, b)
        lengths.append(length)
      
      # print(lengths)

      # 라벨 추가
      lengths.append(1) # 1 - click

      data_click_ver3_1 = data_click_ver3_1.append([lengths])

In [15]:
data_click_ver3_1.head()

,0,1,2,3
0,0.123421,0.348014,0.375801,1
0,0.127347,0.346963,0.376199,1
0,0.125835,0.347519,0.376901,1
0,0.123494,0.346019,0.376633,1
0,0.124907,0.347254,0.376921,1


In [16]:
data_click_ver3_1.tail()

,0,1,2,3
0,0.084860,0.250267,0.313357,1
0,0.088588,0.252296,0.315560,1
0,0.087665,0.250667,0.311527,1
0,0.088363,0.252362,0.313507,1
0,0.088911,0.253370,0.316330,1


In [17]:
data_click_ver3_1.to_csv("dataset_click_ver3_1.csv")

#### 클릭이 아닌 모션

In [19]:
cap = cv2.VideoCapture('none.mov') # video open

data_none_ver3_1 = pd.DataFrame()

while cap.isOpened():
  ret, img = cap.read()

  if img is None:
    break

  img = cv2.flip(img, 1)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  result = hands.process(img)
  img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

  index_point = [[8, 5], [8, 0], [7, 0]]
  point_info = np.zeros((21, 3))
  if result.multi_hand_landmarks is not None:
    for res in result.multi_hand_landmarks:
      
      for j, lm in enumerate(res.landmark):
        point_info[j] = [lm.x, lm.y, lm.z]
      
      lengths = []
      for point in index_point:
        # 두 점 사이 거리 계산
        a = (point_info[point[0]][0], point_info[point[0]][1], point_info[point[0]][2]) # point[0]
        b = (point_info[point[1]][0], point_info[point[1]][1], point_info[point[1]][2]) # point[1]
        length = distance.euclidean(a, b)
        lengths.append(length)
      
      # print(lengths)

      # 라벨 추가
      lengths.append(0) # 0 - click

      data_none_ver3_1 = data_none_ver3_1.append([lengths])

In [20]:
data_none_ver3_1.head()

,0,1,2,3
0,0.243827,0.490555,0.429470,0
0,0.238780,0.495131,0.432374,0
0,0.230851,0.484009,0.423994,0
0,0.233743,0.483161,0.422337,0
0,0.230710,0.479666,0.419878,0


In [21]:
data_none_ver3_1.tail()

,0,1,2,3
0,0.225320,0.458669,0.400109,0
0,0.225807,0.460271,0.401534,0
0,0.227421,0.459310,0.400316,0
0,0.230247,0.463203,0.403559,0
0,0.227566,0.461257,0.402396,0


In [22]:
data_none_ver3_1.to_csv("dataset_none_ver3_1.csv")